https://gist.github.com/rafguns/faff8dc090b67a783b85d488f88952ba?short_path=fb013e3

# Confidence intervals for weighted cosine similarity

This document provides a detailed overview of our bootstrap-based approach to determining confidence intervals for weighted cosine similarity. It is a companion to the [guide that focuses on distance between barycenters](http://nbviewer.jupyter.org/gist/rafguns/6fa3460677741e356538337003692389). While the procedure in both cases is very similar, it is not exactly the same. Parts of this document are based on or taken from the barycenter guide.

To make it easier to follow, we will use a small hypothetical example, determining the confidence interval for simialrity between two sets of publications. As in the barycenter guide, the implementation is in Python.

First we load some libraries.

In [1]:
import numpy as np
import pandas as pd

## Weighted cosine similarity

### Getting the data ready

Now suppose that we have a panel member `pm` and a research group `group`. For both, we have counted their number of publications in each Web of Science Subject Category (SC). For instance, `pm` has 65 publications in *Chemistry, Inorganic Nuclear*, 40 in *Chemistry, Organic*, and so on.

In [2]:
pm = pd.Series({
    'CHEMISTRY, INORGANIC & NUCLEAR': 65,
    'CHEMISTRY, ORGANIC': 40,
    'CHEMISTRY, MULTIDISCIPLINARY': 15,
    'CRYSTALLOGRAPHY': 4,    
})

group = pd.Series({
    'CHEMISTRY, MULTIDISCIPLINARY': 122,
    'BIOPHYSICS': 89,
    'CHEMISTRY, ORGANIC': 45,
    'CHEMISTRY, PHYSICAL': 42,
    'OPTICS': 26,
    'ELECTROCHEMISTRY': 14,
    'MICROSCOPY': 3,
})

Apart from the publication profiles, we also need a similarity matrix. This is a symmetric matrix $S$ in which each element $S_{i,j}$ expresses the similarity between SCs $i$ and $j$. For this example we load a similarity matrix that is freely available on [Loet Leydesdorff's website](http://www.leydesdorff.net/overlaytoolkit/). Of course, other similarity matrices might yield different results.

In [3]:
matrix = pd.read_excel('http://www.leydesdorff.net/overlaytoolkit/matrix10.xlsx', sheetname='Cosin Sim Citing')

Since the matrix has 224 rows and columns, it is too large to display. We can, however, get an idea of what is inside by looking at the first few rows.

In [4]:
matrix.head()

,ACOUSTICS,AGRICULTURAL ECONOMICS & POLICY,AGRICULTURAL ENGINEERING,"AGRICULTURE, DAIRY & ANIMAL SCIENCE","AGRICULTURE, MULTIDISCIPLINARY",AGRONOMY,ALLERGY,ANATOMY & MORPHOLOGY,ANDROLOGY,ANESTHESIOLOGY,...,TRANSPORTATION,TRANSPORTATION SCIENCE & TECHNOLOGY,TROPICAL MEDICINE,URBAN STUDIES,UROLOGY & NEPHROLOGY,VETERINARY SCIENCES,VIROLOGY,WATER RESOURCES,WOMEN,ZOOLOGY
ACOUSTICS,1.000000,0.007630,0.038938,0.018006,0.025958,0.013559,0.017154,0.097334,0.090629,0.057525,...,0.007886,0.102819,0.022850,0.001234,0.053652,0.024052,0.024069,0.045031,0.007219,0.044930
AGRICULTURAL ECONOMICS & POLICY,0.007630,1.000000,0.060740,0.069574,0.127465,0.047864,0.006178,0.013845,0.017045,0.003245,...,0.096653,0.048697,0.014034,0.326270,0.005122,0.031746,0.010665,0.065005,0.050875,0.036458
AGRICULTURAL ENGINEERING,0.038938,0.060740,1.000000,0.159571,0.444598,0.319055,0.031748,0.115049,0.090755,0.011790,...,0.010778,0.066520,0.106832,0.004900,0.024656,0.122964,0.190731,0.494796,0.001752,0.109991
"AGRICULTURE, DAIRY & ANIMAL SCIENCE",0.018006,0.069574,0.159571,1.000000,0.412862,0.121670,0.049027,0.184242,0.246865,0.021821,...,0.004274,0.002057,0.099812,0.000707,0.040783,0.560081,0.130650,0.029649,0.005266,0.166784
"AGRICULTURE, MULTIDISCIPLINARY",0.025958,0.127465,0.444598,0.412862,1.000000,0.614654,0.056437,0.159252,0.132730,0.025192,...,0.007933,0.014974,0.110815,0.005229,0.043964,0.194955,0.156331,0.191959,0.008421,0.170428


One can see that similarity values range between zero and one.

At the moment, the `pm` and `group` data only contain counts for the SCs in which they have actually published. We will now add zeroes for all the other SCs by copying them from the matrix. This way, the matrix, `pm` and `group` all carry information on the exact same set of SCs.

In [5]:
pm = pm.reindex(matrix.index, fill_value=0).astype(int)
group = group.reindex(matrix.index, fill_value=0).astype(int)

### Determining weighted cosine similarity

Weighted cosine similarity (WCS) was proposed by [Zhou et al. (2012)](http://doi.org/10.1007/s11192-012-0767-9). The WCS between a panel member, represented by publication vector $M$, and a research group, represented by publication vector $R$, is defined as:

$$WCS(M, R) = \frac{\sum_{i=1}^N M_i (\sum_{j=1}^N R_j S_{ji})}
                   {\sqrt{\sum_{i=1}^N M_i (\sum_{j=1}^N M_j S_{ji})} \cdot \sqrt{\sum_{i=1}^N R_i (\sum_{j=1}^N R_j S_{ji})}}$$
                   
Written as matrix operations, this becomes:

$$WCS(M, R) = \frac{M^t \cdot S \cdot R}{\sqrt{M^t \cdot S \cdot M} \cdot \sqrt{R^t \cdot S \cdot R}}$$

where $^t$ denotes matrix transposition and $S$ is the similarity matrix. If the similarity matrix $S$ is the identity matrix, the above equation reduces to regular cosine similarity. We can write a Python fuction to compute WCS using matrix operations:

In [6]:
def weighted_cosine_sim(M, R, S):
    # because of the way one-dimensional arrays work, explicit transposition is not necessary
    return M.dot(S).dot(R) / np.sqrt(M.dot(S).dot(M) * R.dot(S).dot(R))

Using this function we obtain the WCS between `pm` and `group`.

In [7]:
weighted_cosine_sim(pm, group, matrix)

0.79151248916360006

We find a WCS of about 0.79. Note that this is higher than one might expect based on the limited overlap in SCs between `pm` and `group`. The reason for the high similarity is that many of the SCs are very similar themselves.

For comparison, the regular cosine similarity is only 0.28: 

In [8]:
weighted_cosine_sim(pm, group, np.identity(len(pm)))

0.28118138870339504

## Bootstrapped confidence intervals


### Bootstrap sample

We now turn to the problem of estimating the stability of the value we found. As in the barycenter guide, we do this using a bootstrapping approach.

Bootstrapping is a simulation-based method for estimating standard error and confidence intervals. Bootstrapping depends on the notion of a **bootstrap sample**. To determine a bootstrap sample for a panel member or research group with N publications, we randomly sample with replacement N publications from its set of publications. In other words, the same publication can be chosen multiple times. Some publications in the original data set will not occur in the bootstrap data set, whereas others will occur once, twice or even more times.

In the following small example, there are six papers, published in respectively SC 2, SC 1, SC 3, SC 2, SC 3 and SC 5.

In [9]:
example = np.array([2, 1, 3, 2, 3, 5])

A bootstrap sample for this example is a new array with the same size (six elements), sampled from the original data. We sample with replacement, meaning that the same paper can be picked more than once.

In [10]:
def bootstrap_sample_from_papers(papers):
    return np.random.choice(papers, papers.size, replace=True)

Because SCs 2 and 3 occur twice in the original data, these are also more likely to be chosen. We call `bootstrap_sample_from_papers` a few times to show how it samples from the input data.

In [11]:
bootstrap_sample_from_papers(example)

array([1, 3, 2, 2, 3, 3])

In [12]:
bootstrap_sample_from_papers(example)

array([1, 1, 2, 5, 5, 5])

In [13]:
bootstrap_sample_from_papers(example)

array([3, 3, 2, 2, 1, 5])

However, our data are in a form that is centered around SCs rather than individual papers: each row of `pm` and `group` represents a SC and the number of papers in it. We therefore translate from the SC-centered representation to the paper-centered one that is expected by `bootstrap_sample_from_papers`. Once we have a sample, we translate it back to a SC-centered representation.

In [14]:
def bootstrap_sample_from_categories(num_papers_per_category):
    # First, transform num_papers_per_category into a paper array
    # where each different value denotes a different SC.
    n_categories = num_papers_per_category.size
    papers = np.repeat(np.arange(n_categories), num_papers_per_category)

    # Draw sample from papers
    sample = bootstrap_sample_from_papers(papers)

    # Count number of papers in each SC
    return np.bincount(sample, minlength=n_categories)

To illustrate the kind of output this gives, we call it a few times with a fictional example: a list of SCs, in which we have respectively 5, 0, 4, 10, 0, and 0 papers. Note that the total number of papers (19) stays constant.

In [15]:
example_data = np.array([5, 0, 4, 10, 0, 0])
bootstrap_sample_from_categories(example_data)

array([7, 0, 3, 9, 0, 0], dtype=int64)

In [16]:
bootstrap_sample_from_categories(example_data)

array([6, 0, 4, 9, 0, 0], dtype=int64)

Now we can determine a *bootstrap replication*, the statistic we are interested in, starting not from the original data but from the bootstrap sample data. Below, we obtain a bootstrap sample for `pm` and `group` and consequently the corresponding bootstrap replication of their similarity.

In [17]:
pm_sample = bootstrap_sample_from_categories(pm)
group_sample = bootstrap_sample_from_categories(group)

weighted_cosine_sim(pm_sample, group_sample, matrix)

0.76507180844009237

The value we obtain is slightly lower than the similarity based on the empirically found counts (0.79).

### Confidence intervals

One or a few bootstrap samples are not enough to estimate a confidence interval. For that, it is recommended to obtain 1000 or more bootstrap samples. From each sample we calculate the bootstrap replication.

In our case, we calculate 1000 bootstrapped similarity values.

In [18]:
replication_count = 1000
replications = np.empty(replication_count)

for i in range(replication_count):
    pm_sample = bootstrap_sample_from_categories(pm)
    group_sample = bootstrap_sample_from_categories(group)

    replications[i] = weighted_cosine_sim(pm_sample, group_sample, matrix)

Let's look at the first ten replications.

In [19]:
replications[:10]

array([ 0.80221476,  0.76605643,  0.79051016,  0.82246141,  0.7661963 ,
        0.80273074,  0.78869054,  0.78316525,  0.79785403,  0.78274641])

As we expected, they all seem to lie fairly close to the WCS which was based on the empirical data. Below we show the minimum, maximum, median and mean.

In [20]:
print('Minimum:', np.min(replications))
print('Maximum:', np.max(replications))
print('Median:', np.median(replications))
print('Mean:', np.mean(replications))

Minimum: 0.730969905648
Maximum: 0.847106600479
Median: 0.790739216682
Mean: 0.7904931757


Finally, to obtain a 95% confidence interval, we use **bootstrap percentiles** (other ways of determining confidence intervals in bootstrapping also exist). We first sort the distances from small to large. The lower and upper bound of the confidence interval are then (in this case) the 25th and 975th distance, such that 95% of the variation is within the interval.

More generally, for a $1 - \alpha$ interval, the lower and upper bound are found at $\frac{N \alpha}{2}$ and $\frac{N (1 - \alpha)}{2}$, respectively, where $N$ denotes the number of samples.

In [21]:
replications = np.sort(replications)
lower = replications[25]
upper = replications[975]

print('Lower =', lower)
print('Upper =', upper)

Lower = 0.756301798053
Upper = 0.82287302336


In summary, for this example we obtain a confidence interval between (roughly) 0.76 and 0.82.